Change the runtime to GPU T4 then connect.
Run the below cell to connect to your google drive, where you uploaded qgis_copilot folder at the root of your drive.




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Run the following commands in the terminal, not in the cells.

Open terminal and run following command to setup
the micromamba environment for qgis:

```
cd drive/MyDrive/qgis_copilot
chmod +x setup.sh
./setup.sh
```

How to find elev_bands in dem:
```
chmod +x run_elev_bands.sh
./run_elev_bands.sh proj/data/dem_filled.tif proj/data/elev_bands_5m.gpkg 5
```


Steps for setting up LLM model.
Note: All code are needed to be executed in terminal of colab

Open terminal and run your codes listed below there:

0) In Colab or your terminal: point to micromamba
```
export PATH=/opt/mamba/bin:$PATH
```

1) Create a clean env (example: Python 3.10)
```
micromamba create -y -n qgis310 python=3.10
micromamba run -n qgis310 python -V
```

2) Install PyTorch that matches your CUDA (example: CUDA 12.x wheels)
```
# Use the command from https://pytorch.org/get-started/locally/
micromamba run -n qgis310 pip install torch --index-url https://download.pytorch.org/whl/cu121
```

3) Install vLLM matched to that Torch/CUDA, plus the rest
```
micromamba run -n qgis310 pip install vllm transformers accelerate huggingface_hub openai
```

4) Download the model:
```
micromamba run -n qgis310 python - <<'PY'
from huggingface_hub import snapshot_download
print("⬇️  Qwen2-7B-Instruct-AWQ …")
p = snapshot_download("Qwen/Qwen2-7B-Instruct-AWQ",
                      local_dir="./models/qwen2-7b-instruct-awq")
print("MODEL_DIR="+p)
PY
```

5) Run server in the terminal:
```
MODEL_DIR="./models/qwen2-7b-instruct-awq"
micromamba run -n qgis310 python -m vllm.entrypoints.openai.api_server \
  --model "$MODEL_DIR" --quantization awq \
  --host 127.0.0.1 --port 8000 --dtype auto --max-model-len 8192
```
  
6) Then run rest of the Notebook cells.

Once server is running in the terminal and it shows:
```
(APIServer pid=6785) INFO:     Started server process [6785]
(APIServer pid=6785) INFO:     Waiting for application startup.
(APIServer pid=6785) INFO:     Application startup complete.
```
Then run below cell.

In [ ]:
from openai import OpenAI
client = OpenAI(base_url="http://127.0.0.1:8000/v1", api_key="not-needed")

# See the actual model id(s)
models = client.models.list()
print([m.id for m in models.data])  # e.g. ['./models/qwen2-7b-instruct-awq']

# Use the exact id returned above
mid = models.data[0].id
resp = client.chat.completions.create(
    model=mid,
    messages=[{"role":"user","content":"Say hello in one short line."}],
)
print(resp.choices[0].message.content)

['./models/qwen2-7b-instruct-awq']
Hello!


Automatically checks if the QGIS tools server is running on 127.0.0.1:7000, and if not, starts it under your micromamba env and waits until it’s healthy.


Installing FastAPI

In [ ]:
import subprocess, sys, os, time, requests, pathlib

MICRO="/opt/mamba/bin/micromamba"
ENV="qgis310"

subprocess.run([MICRO,"run","-n",ENV,"python","-m","pip","install","-q",
                "fastapi","uvicorn[standard]","pydantic","requests"], check=True)
print("✅ FastAPI/uvicorn installed in qgis310")


✅ FastAPI/uvicorn installed in qgis310


Creating qgis server py file

In [ ]:
code = r"""
import os, json
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

# ---- QGIS bootstrap (safe even if QGIS is absent) ----
PREFIX = os.environ.get("QGIS_PREFIX_PATH") or os.sys.prefix
os.environ.setdefault("QT_QPA_PLATFORM","offscreen")
os.environ.setdefault("QGIS_PREFIX_PATH", PREFIX)
os.environ.setdefault("QGIS_PLUGINPATH", os.path.join(PREFIX,"lib","qgis","plugins"))
os.environ.setdefault("PROJ_LIB", os.path.join(PREFIX,"share","proj"))
os.environ.setdefault("GDAL_DATA", os.path.join(PREFIX,"share","gdal"))
os.makedirs("/tmp/runtime-qgis", exist_ok=True)
os.environ.setdefault("XDG_RUNTIME_DIR","/tmp/runtime-qgis")

QGIS_OK, QGIS_VER, QGIS_ERR = False, "unknown", None
try:
    from qgis.core import QgsApplication, Qgis
    from processing.core.Processing import Processing
    import processing
    app_q = QgsApplication([], False)
    app_q.setPrefixPath(PREFIX, True)
    app_q.initQgis()
    Processing.initialize()
    QGIS_OK, QGIS_VER = True, str(Qgis.QGIS_VERSION)
except Exception as e:
    QGIS_ERR = repr(e)

# ---- Replace this with your real elev-band processing.run(...) ----
def make_elev_bands(args):
    in_tif   = args.get("in_tif")
    interval = int(args.get("interval", 5))
    out_gpkg = args.get("out_gpkg","/content/elev_bands.gpkg")
    if not QGIS_OK:
        return {"ok": False, "error": f"QGIS not ready: {QGIS_ERR}", "input": in_tif, "interval": interval, "output": out_gpkg}
    # Example skeleton (uncomment and adapt):
    # params = {
    #   "INPUT": in_tif,
    #   "INTERVAL": interval,
    #   "FIELD_NAME": "ELEV",
    #   "CREATE_3D": False,
    #   "IGNORE_NODATA": True,
    #   "NODATA": None,
    #   "OFFSET": 0,
    #   "OUTPUT": out_gpkg,
    # }
    # processing.run("gdal:contour", params)
    return {"ok": True, "input": in_tif, "interval": interval, "output": out_gpkg}

# ---- API wiring ----
class ToolReq(BaseModel):
    name: str
    args: dict = {}

app = FastAPI()

@app.get("/health")
def health():
    return {"qgis_ready": QGIS_OK, "qgis_version": QGIS_VER, "qgis_error": QGIS_ERR}

@app.post("/tool")
def tool(req: ToolReq):
    if req.name == "make_elev_bands":
        return make_elev_bands(req.args)
    return {"error": f"unknown tool {req.name}"}

if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=7000)
"""
path = pathlib.Path("/content/qgis_tools_server.py")
path.write_text(code)
print("✅ Wrote /content/qgis_tools_server.py")


✅ Wrote /content/qgis_tools_server.py


Starting QGIS server

In [ ]:
MICRO="/opt/mamba/bin/micromamba"; ENV="qgis310"

# start in background
proc = subprocess.Popen([MICRO,"run","-n",ENV,"python","/content/qgis_tools_server.py"],
                        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# wait up to ~10s for /health
import time, requests
ok=False
for _ in range(20):
    time.sleep(0.5)
    try:
        r = requests.get("http://127.0.0.1:7000/health", timeout=1)
        print("Health:", r.json()); ok=True; break
    except Exception:
        pass

# if not up, print first 60 lines of logs to see why
if not ok:
    print("❌ Server didn’t start; showing logs:")
    for i, line in zip(range(60), proc.stdout):
        print(line.rstrip())

Health: {'qgis_ready': True, 'qgis_version': '3.28.12-Firenze', 'qgis_error': None}


Moment of truth:

Here note down that the chat prompt is under:



```
resp = client.chat.completions.create(
    model=mid,
    messages=[
      {"role":"system","content":"Use the provided arguments exactly when calling the function."},
      {"role":"user","content": f"Call make_elev_bands with these args: {json.dumps(call_args)}"}
    ],
    tools=TOOLS,
    tool_choice={"type":"function","function":{"name":"make_elev_bands"}}
)
```




In [ ]:
from openai import OpenAI
import json, requests

BASE = "http://127.0.0.1:8000/v1"   # current server (no auto tool-choice)
client = OpenAI(base_url=BASE, api_key="not-needed")

mid = client.models.list().data[0].id  # use served id (often the model path)

TOOLS = [{
  "type":"function",
  "function":{
    "name":"make_elev_bands",
    "description":"Generate elevation bands with QGIS.",
    "parameters":{
      "type":"object",
      "properties":{
        "in_tif":{"type":"string"},
        "interval":{"type":"integer"},
        "out_gpkg":{"type":"string"}
      },
      "required":["in_tif"]
    }
  }
}]

# 👉 set your real paths here
call_args = {"in_tif":"/content/drive/MyDrive/qgis_copilot/proj/data/dem_filled.tif", "interval":5, "out_gpkg":"/content/elev_bands_5m.gpkg"}

resp = client.chat.completions.create(
    model=mid,
    messages=[
      {"role":"system","content":"Use the provided arguments exactly when calling the function."},
      {"role":"user","content": f"Call make_elev_bands with these args: {json.dumps(call_args)}"}
    ],
    tools=TOOLS,
    tool_choice={"type":"function","function":{"name":"make_elev_bands"}}
)

msg = resp.choices[0].message
if not msg.tool_calls:
    # fallback if the model didn't include args—use ours directly
    args = call_args
else:
    args = json.loads(msg.tool_calls[0].function.arguments or "{}")

tool_result = requests.post(
    "http://127.0.0.1:7000/tool",
    json={"name":"make_elev_bands","args":args},
    timeout=600
).json()
print("Tool result:", tool_result)

Tool result: {'ok': True, 'input': '/content/drive/MyDrive/qgis_copilot/proj/data/dem_filled.tif', 'interval': 5, 'output': '/content/elev_bands_5m.gpkg'}


In [ ]:
!ls /content/elev_bands_5m.gpkg


ls: cannot access '/content/elev_bands_5m.gpkg': No such file or directory


In [ ]:
%%file qgis_tools_server.py
# /content/qgis_tools_server.py
import os, json, shlex, subprocess, pathlib
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn

# ---- config: adjust if your paths differ ----
ENV_NAME = os.environ.get("ENV_NAME","qgis310")
MAMBA_ROOT_PREFIX = os.environ.get("MAMBA_ROOT_PREFIX","/opt/mamba")
MAMBA_BIN = os.environ.get("MAMBA_BIN", f"{MAMBA_ROOT_PREFIX}/bin/micromamba")
SCRIPT_PATH = os.environ.get("ELEV_SCRIPT", "/content/drive/MyDrive/qgis_copilot/make_elev_bands.py")

# Ensure micromamba is reachable
if not (os.path.exists(MAMBA_BIN) and os.access(MAMBA_BIN, os.X_OK)):
    # fallback to PATH lookup
    from shutil import which
    MAMBA_BIN = which("micromamba") or MAMBA_BIN

class ToolReq(BaseModel):
    name: str
    args: dict = {}

app = FastAPI()

@app.get("/health")
def health():
    info = {
        "micromamba": MAMBA_BIN,
        "env_name": ENV_NAME,
        "script": SCRIPT_PATH,
    }
    # try to read QGIS version inside the env (best-effort)
    try:
        cmd = [MAMBA_BIN, "run", "-n", ENV_NAME, "python", "-c",
               "from qgis.core import Qgis; print(Qgis.QGIS_VERSION)"]
        ver = subprocess.run(cmd, capture_output=True, text=True, timeout=30)
        info["qgis_version"] = ver.stdout.strip() or ver.stderr.strip()
        info["qgis_status"] = (ver.returncode == 0)
    except Exception as e:
        info["qgis_status"] = False
        info["qgis_error"] = repr(e)
    return info

def _run_make_elev_bands(in_tif: str, interval: int|float = 10, out_gpkg: str|None = None, prefix: str|None = None):
    if not os.path.exists(SCRIPT_PATH):
        raise HTTPException(status_code=500, detail=f"Script not found: {SCRIPT_PATH}")
    if not in_tif or not os.path.exists(in_tif):
        raise HTTPException(status_code=400, detail=f"Input DEM not found: {in_tif}")
    if not out_gpkg:
        out_gpkg = "/content/elev_bands.gpkg"
    os.makedirs(os.path.dirname(out_gpkg), exist_ok=True)

    # Optional: resolve the env’s sys.prefix for --prefix
    env_prefix = None
    try:
        cmd = [MAMBA_BIN, "run", "-n", ENV_NAME, "python", "-c", "import sys; print(sys.prefix)"]
        r = subprocess.run(cmd, capture_output=True, text=True, timeout=20)
        if r.returncode == 0:
            env_prefix = r.stdout.strip()
    except Exception:
        env_prefix = None
    if prefix:  # allow override from args
        env_prefix = prefix

    cmd = [
        MAMBA_BIN, "run", "-n", ENV_NAME, "python", SCRIPT_PATH,
        "--in", in_tif,
        "--out", out_gpkg,
        "--interval", str(interval),
    ]
    if env_prefix:
        cmd += ["--prefix", env_prefix]

    proc = subprocess.run(cmd, capture_output=True, text=True)
    stdout, stderr, rc = proc.stdout, proc.stderr, proc.returncode

    exists = os.path.exists(out_gpkg)
    size = os.path.getsize(out_gpkg) if exists else 0

    result = {
        "ok": (rc == 0 and exists and size > 0),
        "cmd": " ".join(shlex.quote(x) for x in cmd),
        "rc": rc,
        "stdout": stdout[-2000:],  # last 2k chars
        "stderr": stderr[-2000:],
        "input": os.path.abspath(in_tif),
        "interval": float(interval),
        "output": os.path.abspath(out_gpkg),
        "size_bytes": int(size),
        "env_prefix": env_prefix,
    }
    if not result["ok"]:
        raise HTTPException(status_code=500, detail=result)
    return result

@app.post("/tool")
def tool(req: ToolReq):
    if req.name == "make_elev_bands":
        a = req.args or {}
        return _run_make_elev_bands(
            in_tif=a.get("in_tif"),
            interval=a.get("interval", 10),
            out_gpkg=a.get("out_gpkg"),
            prefix=a.get("prefix")
        )
    raise HTTPException(status_code=404, detail=f"Unknown tool: {req.name}")

if __name__ == "__main__":
    uvicorn.run(app, host="127.0.0.1", port=7000)


Overwriting qgis_tools_server.py


In [ ]:
import os, pathlib
os.makedirs("/content/drive/MyDrive/qgis_copilot/proj/data", exist_ok=True)

call_args = {
  "in_tif": "/content/drive/MyDrive/qgis_copilot/proj/data/dem_filled.tif",
  "interval": 5,
  "out_gpkg": "/content/drive/MyDrive/qgis_copilot/proj/data/elev_bands_5m.gpkg",
}


In [ ]:
!fuser -k 7000/tcp || true

7000/tcp:            21003


In [ ]:
MICRO="/opt/mamba/bin/micromamba"; ENV="qgis310"

# start in background
proc = subprocess.Popen([MICRO,"run","-n",ENV,"python","/content/qgis_tools_server.py"],
                        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

# wait up to ~10s for /health
import time, requests
ok=False
for _ in range(20):
    time.sleep(0.5)
    try:
        r = requests.get("http://127.0.0.1:7000/health", timeout=1)
        print("Health:", r.json()); ok=True; break
    except Exception:
        pass

# if not up, print first 60 lines of logs to see why
if not ok:
    print("❌ Server didn’t start; showing logs:")
    for i, line in zip(range(60), proc.stdout):
        print(line.rstrip())

Health: {'micromamba': '/opt/mamba/bin/micromamba', 'env_name': 'qgis310', 'script': '/content/drive/MyDrive/qgis_copilot/make_elev_bands.py', 'qgis_version': '3.28.12-Firenze', 'qgis_status': True}


In [ ]:
from openai import OpenAI
import json, requests, os

BASE = "http://127.0.0.1:8000/v1"
client = OpenAI(base_url=BASE, api_key="not-needed")

mid = client.models.list().data[0].id

TOOLS = [{
  "type":"function",
  "function":{
    "name":"make_elev_bands",
    "description":"Generate elevation bands with QGIS.",
    "parameters":{
      "type":"object",
      "properties":{
        "in_tif":{"type":"string"},
        "interval":{"type":"number"},
        "out_gpkg":{"type":"string"}
      },
      "required":["in_tif"]
    }
  }
}]

call_args = {
  "in_tif": "/content/drive/MyDrive/qgis_copilot/proj/data/dem_filled.tif",
  "interval": 5,
  "out_gpkg": "/content/drive/MyDrive/qgis_copilot/proj/data/elev_bands_5m.gpkg",
}

resp = client.chat.completions.create(
    model=mid,
    messages=[
      {"role":"system","content":"Use the provided arguments exactly when calling the function."},
      {"role":"user","content": f"Call make_elev_bands with these args: {json.dumps(call_args)}"}
    ],
    tools=TOOLS,
    tool_choice={"type":"function","function":{"name":"make_elev_bands"}}
)

msg = resp.choices[0].message
args = call_args if not getattr(msg, "tool_calls", None) else json.loads(msg.tool_calls[0].function.arguments or "{}")

tool_result = requests.post(
    "http://127.0.0.1:7000/tool",
    json={"name":"make_elev_bands","args":args},
    timeout=900
).json()

print("Tool result:", json.dumps(tool_result, indent=2))
out = tool_result.get("output")
print("Exists:", os.path.exists(out), "Size:", os.path.getsize(out) if os.path.exists(out) else 0, "Path:", out)


Tool result: {
  "ok": true,
  "cmd": "/opt/mamba/bin/micromamba run -n qgis310 python /content/drive/MyDrive/qgis_copilot/make_elev_bands.py --in /content/drive/MyDrive/qgis_copilot/proj/data/dem_filled.tif --out /content/drive/MyDrive/qgis_copilot/proj/data/elev_bands_5m.gpkg --interval 5 --prefix /opt/mamba/envs/qgis310",
  "rc": 0,
  "stdout": "DEBUG: prefix: /opt/mamba/envs/qgis310 plugin: /opt/mamba/envs/qgis310/lib/qgis/plugins\nWrote: /content/drive/MyDrive/qgis_copilot/proj/data/elev_bands_5m.gpkg\n",
  "stderr": "",
  "input": "/content/drive/MyDrive/qgis_copilot/proj/data/dem_filled.tif",
  "interval": 5.0,
  "output": "/content/drive/MyDrive/qgis_copilot/proj/data/elev_bands_5m.gpkg",
  "size_bytes": 1429504,
  "env_prefix": "/opt/mamba/envs/qgis310"
}
Exists: True Size: 1429504 Path: /content/drive/MyDrive/qgis_copilot/proj/data/elev_bands_5m.gpkg
